In [ ]:
import tensorflow as tf
import keras

from keras import regularizers
from keras import backend as K

from keras.models import Sequential, Model

from keras.layers import *
from keras.optimizers import *

from keras.callbacks import ModelCheckpoint
from keras.initializers import Constant


from keras.utils import np_utils
from keras_preprocessing.image import ImageDataGenerator


from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.applications import MobileNetV2

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import tensorflow
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from keras.applications import MobileNetV2
import imageio
from keras.utils import to_categorical

In [ ]:
xs = []
ys = []
def Load_data_malignant():
    path ="/content/drive/MyDrive/DATASET/malignant"
    x_out = []
    y_out = []
    for i in range(1, 314):
        img = imageio.imread(path +'/' + '1' + ' (' + str(i) + ')' + '.jpg')
        lab = 1
        x_out.append(img)
        y_out.append(lab)
    return x_out, y_out

In [ ]:
def Load_data_benign():
    path ="/content/drive/MyDrive/DATASET/benign"
    x_out = []
    y_out = []
    for i in range(1, 314):
        img = imageio.imread(path +'/' + '1' + ' (' + str(i) + ')' + '.jpg')
        lab = 0
        x_out.append(img)
        y_out.append(lab)
    return x_out, y_out

In [ ]:
x_0, y_0 = Load_data_benign()

In [ ]:
x_1, y_1 = Load_data_malignant()

In [ ]:
x_0 = np.array(x_0)
y_0 = np.array(y_0)

x_1 = np.array(x_1)
y_1 = np.array(y_1)

In [ ]:
num_classes = 2
x=np.concatenate((x_0, x_1), axis=0)
y=np.concatenate((y_0, y_1), axis=0)

y = np_utils.to_categorical(y, num_classes)

In [ ]:
x_train, x_Test, y_train, y_Test = train_test_split(x, y, test_size=0.3, random_state=5,stratify=y)

In [ ]:
x_train.shape

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=[224,224,3]),
    keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu',name ='feature_dense1'),
    keras.layers.Dense(128, activation='relu',name ='feature_dense2'),
    keras.layers.Dense(64, activation='relu',name ='feature_dense3'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
   ])
model.summary()

In [ ]:
optimizer = keras.optimizers.Adamax()
model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy']

In [ ]:
history=model.fit(x_train, y_train, epochs=150,batch_size = 40 ,validation_data= (x_Test, y_Test), shuffle=True)

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper right')
plt.show()

In [ ]:
model.save('keras_model_conv2D_v2.h5')

In [ ]:
from keras.models import load_model
import tensorflow as tf
import keras

from keras import regularizers
from keras import backend as K

from keras.models import Sequential, Model


model = load_model('keras_model_conv2D.h5')

intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer('feature_dense3').output)
intermediate_layer_model.summary()

In [ ]:
files=[]
for i in range(0,438):
    files.append(train_generator.filenames[i])

filenames=[]
for f in files:
    x=f.replace('\\',"/",1)
    filenames.append('final/train/'+ x)

In [ ]:
new_col = []
for i in range(64):
    new_col.append('Feature_%d'%(i+1))

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
import csv

In [ ]:
preds=[]
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (224, 224, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

for f in filenames:
    image = load(f)
    files.append(f)
    pred=intermediate_layer_model.predict(image)
    preds.append(pred)

df = pd.DataFrame (preds)
filepath = 'new_data.xlsx'

df.to_excel(filepath, index=False)

In [ ]:
preds